In [91]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
import numpy as np

In [92]:
output_folder_path = "../outputs/RealWorld/facebookMedium/bots/02"
input_folder_path="../../output/RealWorld/facebookMedium/bots/02"
#datasets=['astroPh', 'emailEU', 'facebook', 'facebookMedium', 'facebookArtist', 'gitInteractions', 'hepPh', 'lastfm', 'redditHyperlink', 'twitchDE', 'twitchENGB', 'twitchES', 'twitchFR', 'twitchPTBR', 'twitchRU']
#botRatio=['m0_', 'm25_', 'm50_', 'm75_', 'm100_', 'm125_', 'm150_', 'm175_', 'm200_']
datasets=['facebookMedium']
startRatio=['01', '02', '03', '04', '05']
rewireRatio=[ 'r10_', 'r20_', 'r30_', 'r40_', 'r50_', 'r60_', 'r70_', 'r80_', 'r90_']
data2={}
average={}

In [93]:
def create_data(path):
    files=os.listdir(path)
    botRatio=[]
    for file in files:
        if file.endswith(".txt"):
            match = re.search(r"_m(\d+)_", file)
        if match:
            result = match.group(0)
        if not result in botRatio:
            botRatio.append(result)
    botRatio=sorted(botRatio, key=lambda x: int(x.strip('_m')))
    #print(botRatio)
    for rr in rewireRatio:
        data2[rr]={}
        average[rr]={}
        for mat in botRatio:
            data2[rr][mat]=[]
            for file in files:
                if mat in file and rr in file:
                    #print(path+" "+file)
                    with open(os.path.join(path, file), 'r') as f:
                        content = pd.read_csv(path+"/"+file, names=['Epoch', 'Pop', 'Frac', 'DiscEdge'], skiprows=1, sep=" ")
                        content=content.drop(['Epoch', 'Pop'], axis=1)    
                        finalPerc=content.iloc[-1, 0]
                        data2[rr][mat].append(finalPerc)
            avg=sum(data2[rr][mat])/len(data2[rr][mat])
            if mat not in average[rr]: 
                average[rr][mat] = {}
            average[rr][mat]=avg

    print(average)
    # print(data2)

In [94]:
def transform_inner_key(key):
    # Remove the '_m' prefix and '_' suffix
    numeric_part = key.strip("_m_")
    # Divide by 10 to convert to floating-point
    return float(numeric_part) / 10

def transform_outer_key(key):
    numeric_part = key.strip("r_")
    return int(numeric_part)

In [95]:
def create_graphs():
    transformed_data={}
    for k, v in average.items():
        transformed_data[k] = {}
        transformed_data[k] = {transform_inner_key(ik): iv for ik, iv in v.items()}
    final_data = {transform_outer_key(k): v for k, v in transformed_data.items()}
    y_keys = sorted(final_data.keys())  # Outer keys (y-axis)
    x_keys = sorted(final_data[10].keys()) # Inner keys (x-axis)

    color_matrix = np.array([[final_data[y][x] for x in x_keys] for y in y_keys])

    # Plot
    plt.figure(figsize=(6, 6))
    plt.imshow(color_matrix, origin='lower', extent=[min(x_keys), max(x_keys), min(y_keys), max(y_keys)],
            aspect='auto', cmap='viridis')  # Use desired colormap

    # Add colorbar
    plt.colorbar(label='Final Opinion Density of Minority')

    # Label axes
    plt.xlabel('Bots Added (%)')
    plt.ylabel('Rewire Rate (%)')
    plt.title('Phase Plot with Minority Opinion Density Starting at 20%')

    # Show the plot
    figpath=output_folder_path+"/PhasePlot.png"
    plt.savefig(figpath, dpi=350)
    plt.show()

In [1]:
for dataset in datasets:
    create_data(input_folder_path)
    create_graphs()

NameError: name 'datasets' is not defined